In [1]:
%scala
import java.net.URL
import java.io.File
import org.apache.commons.io.FileUtils
import org.apache.spark.sql.functions._
import scala.util.Try

dbutils.fs.unmount("/mnt/test")
dbutils.fs.mount(
  source = "wasbs://mag-2019-01-25@iunimag.blob.core.windows.net/",
  mountPoint = "/mnt/test",
  extraConfigs = Map("fs.azure.account.key.iunimag.blob.core.windows.net" -> dbutils.secrets.get(scope = "AzureKeyVaultTest", key = "blobKeyDatabricks")))


//val df = spark.read.parquet("adl://iunimag.azuredatalakestore.net/WoS/CORE_2014_2015_Annuals")
display(dbutils.fs.ls("/mnt/test"))

path,name,size
dbfs:/mnt/test/AIpapersAll.tsv,AIpapersAll.tsv,933089982
dbfs:/mnt/test/AIpapersCitationsAll.tsv,AIpapersCitationsAll.tsv,8090967470
dbfs:/mnt/test/AIpapersExtend.tsv,AIpapersExtend.tsv,7752154999
dbfs:/mnt/test/AIpapersOthers.tsv,AIpapersOthers.tsv,7402542807
dbfs:/mnt/test/AIpapersPatent.tsv,AIpapersPatent.tsv,157413931
dbfs:/mnt/test/AIpapersWoS.csv.gz,AIpapersWoS.csv.gz,301064055
dbfs:/mnt/test/KatzData2.tar,KatzData2.tar,443852800
dbfs:/mnt/test/MAGwosJournalMatch/,MAGwosJournalMatch/,0
dbfs:/mnt/test/PMwosCitations.csv.gz,PMwosCitations.csv.gz,3186720390
dbfs:/mnt/test/YangyangExtra.csv.gz,YangyangExtra.csv.gz,1633659711


In [2]:
import pyspark.sql.functions as sql
df = spark.read.format("parquet").load("/mnt/test/paperContext/*.parquet")
df.describe().show()

+-------+--------------------+--------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+---------------+-------------+--------------------+------------------+--------------------+-------+--------------------+------------------+--------------------+--------------------+------------------+------------------+-----------------+--------------------+
summary| AuthorIds| AuthorNames| AuthorOrders| OrigAffiliates| Affiliates| FoSNames| JournalId| JNormalizedName| JDisplayName|ConferenceInstanceId|CNormalizedName| CDisplayName| PaperId| Rank| Doi|DocType| OriginalTitle| Year| Date| Publisher| ReferenceCount| CitationCount|EstimatedCitation| IndexedAbstract|
+-------+--------------------+--------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+---------------+-------------+--------------------+------------------+--------------------+-------+--------------------+------------------+--------------------+--------------------+------------------+------------------+-----------------+--------------------+
 count| 11002296| 11002296| 11002296| 10337896| 10120595| 11002291| 7486005| 7486005| 7486005| 599571| 599571| 599571| 11002291| 11002291| 8938386|9118303| 11002291| 11002195| 11002195| 9286421| 11002192| 11002192| 11002192| 10514959|
 mean| 2.200472894556975E9| null| 1.0012911857448987| 21727.75| 2.534630048974547E8| null|4.159837153471004E8| null| Infinity|1.0341077435288181E9| null| null|1.9358224227043462E9| 21705.49013291868| null| null| null|2006.9805420645607| null| Infinity| 27.68500776936087| 25.49891476171294|36.39090819356725| null|
 stddev| 6.230952793491633E8| null|0.041121691466681626| 35529.68074136515| 4.661234496232324E8| null|8.396901271491109E8| null| NaN|1.1870855556173432E9| null| null| 5.16527849233918E8|1571.6786119406877| null| null| null| 7.789298308426898| null| NaN|29.910236336909605|115.65536945624177|188.3672872140101| null|
 min|100004360; 132046...| !Chi-jung Fu| 1|Gran Sasso Scien...| 100005738|0 10 v lighting c...| 100000117| 3c empresa| 3C Empresa| 100027508| 3dtv-con 2007|3DTV-CON 2007| 1000002739| 11915|10.0000/anziamj.v...| Book| 'How's My Doctor...| 1830|1830-01-01T00:00:...|"1 Decembrie 1918...| 1| 0| 0|{"IndexLength":1,...|
 max|99998617; 1834048...|林士堯; 邱柏訊; Po-Hsun...|9; 9; 9; 8; 8; 8;...|𝒢ℝεℂ𝒪, Institut...|; ; ; ; ; ; ; ; ;...|ω automaton; virt...| 999906910|žurnal institucio...|Žurnal Institucio...| 99984635| zeus 2012| ssme 2010| 9999997| 36615|10.9999/icm2018-v...| Patent|￼Recidivism Risk ...| 2019|2019-07-01T00:00:...|�The American Phy...| 999| 9995| 999|{"IndexLength":99...|
+-------+--------------------+--------------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+---------------+-------------+--------------------+------------------+--------------------+-------+--------------------+------------------+--------------------+--------------------+------------------+------------------+-----------------+--------------------+

In [3]:
from pyspark.sql.functions import desc

display(df.groupBy("Publisher").count().orderBy('count', ascending=False).where(sql.col("Publisher").isNotNull()).limit(30))

Publisher,count
IEEE,1255371
Elsevier,766429
Springer-Verlag,316756
"Springer, Berlin, Heidelberg",292862
BioMed Central,218540
Taylor & Francis Group,214818
Oxford University Press,194088
Springer US,165955
Springer Netherlands,163853
ACM,155207


In [4]:
display(df.groupBy("JDisplayName").count().orderBy('count', ascending=False).where(sql.col("JDisplayName").isNotNull()).limit(30))

JDisplayName,count
PLOS ONE,91565
Journal of Geophysical Research,25303
Journal of High Energy Physics,19589
The Journal of Neuroscience,17603
The Journal of Infectious Diseases,15800
Cancer Research,15284
International Journal of Computer Applications,15175
Geophysical Research Letters,14749
Hydrobiologia,13165
Physical Review D,12622


In [5]:
display(df.groupBy("Year").count().orderBy('count', ascending=False).where(sql.col("Year").isNotNull()).limit(30))

Year,count
2011,786310
2010,777235
2014,764616
2009,746007
2013,730751
2012,710313
2008,688229
2015,675424
2007,616657
2006,559498


In [6]:
from pyspark.sql.functions import split, explode 

FoS = df.withColumn("FoSlist", explode(split(sql.col("FoSNames"), ";")))
FoS.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+---------------+------------+----------+-----+--------------------+-------+--------------------+----+--------------------+------------+--------------+-------------+-----------------+--------------------+--------------------+
 AuthorIds| AuthorNames| AuthorOrders| OrigAffiliates| Affiliates| FoSNames|JournalId| JNormalizedName| JDisplayName|ConferenceInstanceId|CNormalizedName|CDisplayName| PaperId| Rank| Doi|DocType| OriginalTitle|Year| Date| Publisher|ReferenceCount|CitationCount|EstimatedCitation| IndexedAbstract| FoSlist|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+---------------+------------+----------+-----+--------------------+-------+--------------------+----+--------------------+------------+--------------+-------------+-----------------+--------------------+--------------------+
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| arabidopsis|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| gene silencing|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| cell biology|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| protein protein ...|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| genetics|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| epigenetics|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| arabidopsis thal...|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| biology|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| cytosol|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| bimolecular fluo...|
 2183624072|Melanie Sarah Montes| 1| null| null|arabidopsis; gene...| null| null| null| null| null| null|2187727481|25200| null| null|Protein-protein i...|2011|2011-01-01T00:00:...| null| 24| 0| 0|{"IndexLength":23...| gene expression|
2175747903; 21281...|Terence Z. Wong; ...|4; 8; 7; 2; 1; 5;...|North Carolina Sc...|; 170897317; 1708...|breast cancer; bi...| 13860802|journal of surgic...|Journal of Surgic...| null| null| null|1958441549|20872| 10.1002/jso.23915|Journal|Impact of delayed...|2015|2015-

In [7]:
from pyspark.sql.functions import countDistinct
display(FoS.groupBy("FoSlist").agg(countDistinct("PaperId")).orderBy('count(DISTINCT PaperId)', ascending=False).where(sql.col("FoSlist").isNotNull()).limit(30))

FoSlist,count(DISTINCT PaperId)
computer science,1619107
biology,1612313
medicine,1331309
mathematics,1275740
artificial intelligence,692636
psychology,613695
engineering,603406
chemistry,560303
internal medicine,536163
physics,517637
